In [35]:
# 1. Import Necessary Libraries #this is copied from the sample code provided by btt
import pandas as pd
import numpy as np
import setuptools.dist
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout, Input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import xgboost as xgb
import seaborn as sns

# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network

In [36]:
# 2. Load Data #also copied from btt sample
train_df = pd.read_csv('bttai-ajl-2025/train.csv')
test_df = pd.read_csv('bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

In [37]:
train_df.head() 

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,NaN,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,1 Diagnostic,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,NaN,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,NaN,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...


In [38]:
test_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,qc,ddi_scale
0,0844ae634f0e6e7ef1f73c2aeecbae0e.jpg,2,2,NaN,12
1,3b290d262098f761d719aa07cf36c040.jpg,4,3,NaN,34
2,cf561d08ac46d0fda678bff6621005ee.jpg,2,3,NaN,12
3,e6371069be05c6b0a95b4b3f1bacc9a5.jpg,4,3,NaN,34
4,f76cddb37265f97508f159078dcc7e7c.jpg,5,5,NaN,56


In [39]:
print(test_df.columns) #to check the columns in df's
print(train_df.columns)

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'qc',
       'ddi_scale'],
      dtype='object')
Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'ddi_scale',
       'file_path'],
      dtype='object')


In [40]:
#some directory info to be used later
data_dir = "bttai-ajl-2025/train/train/"
test_dir = "bttai-ajl-2025/test/test/"
pre_built_model=keras.models.load_model('na_model.keras') 
#load the model. im using nasrin's cnn as a base, but feel free to supplment with another cnn trained for the dataset

In [41]:
from sklearn.preprocessing import LabelEncoder #to encode the labels. important that this is done before splitting the data
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

In [42]:
'''
# Function to balance classes
def balance_classes(df, class_col):
    max_size = df[class_col].value_counts().max()
    df_balanced = df.groupby(class_col).apply(lambda x: resample(x, replace=True, n_samples=max_size, random_state=123)).reset_index(drop=True)
    return df_balanced
#theres a class imbalance, so we will balance the classes using resampling (deterministic by using random_state)
train_df = balance_classes(train_df, 'label') '
'''

"\n# Function to balance classes\ndef balance_classes(df, class_col):\n    max_size = df[class_col].value_counts().max()\n    df_balanced = df.groupby(class_col).apply(lambda x: resample(x, replace=True, n_samples=max_size, random_state=123)).reset_index(drop=True)\n    return df_balanced\n#theres a class imbalance, so we will balance the classes using resampling (deterministic by using random_state)\ntrain_df = balance_classes(train_df, 'label') '\n"

In [43]:
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42) #train test split on training data. using 20% of the data for validation


train_datagen = ImageDataGenerator(rescale=1./255) #rescaling the images to [0,1] range
val_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
train_data #exploratory stuffs

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path,encoded_label
1904,ae5dbb5d80c5ccff3104ffe0221639e8.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,1 Diagnostic,12,basal-cell-carcinoma-morpheiform/ae5dbb5d80c5c...,4
2053,901837baf231c9f21227066c15513974.jpg,5,3,acne,inflammatory,non-neoplastic,NaN,56,acne/901837baf231c9f21227066c15513974.jpg,0
651,0d7c7bd5304554919e0179209ef6f1bf.jpg,2,1,keloid,inflammatory,non-neoplastic,NaN,12,keloid/0d7c7bd5304554919e0179209ef6f1bf.jpg,12
2221,1403b063b54423d4f1f85ad7e6504ec1.jpg,2,2,dermatomyositis,inflammatory,non-neoplastic,NaN,12,dermatomyositis/1403b063b54423d4f1f85ad7e6504e...,6
2044,65f7b7df48effe3ad7238404ca5318ce.jpg,4,3,melanoma,malignant-melanoma,malignant,NaN,34,melanoma/65f7b7df48effe3ad7238404ca5318ce.jpg,14
...,...,...,...,...,...,...,...,...,...,...
1638,d34976d2cebb41b26f8cc2bec26f6b64.jpg,2,3,dermatofibroma,benign-dermal,benign,NaN,12,dermatofibroma/d34976d2cebb41b26f8cc2bec26f6b6...,5
1095,7fd141e0b81e5573612d46472ec9fe52.jpg,2,3,melanoma,malignant-melanoma,malignant,NaN,12,melanoma/7fd141e0b81e5573612d46472ec9fe52.jpg,14
1130,d4503411ed629b1252df949d73aa92ff.jpg,4,3,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/d4503411ed629b1252df949d73aa...,16
1294,41169ebef5fa15937f0eefb98d81339c.jpg,4,3,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/41169ebef5fa15937f0eefb98d81...,16


In [45]:
val_data #more exploration

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path,encoded_label
1586,3be5d580b67406152ee65b85d465db27.jpg,4,5,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/3be5d580b67406152ee65b85d465...,16
1748,6cba1a3c302571165ea1d3f81f0ddf5b.jpg,2,2,eczema,inflammatory,non-neoplastic,NaN,12,eczema/6cba1a3c302571165ea1d3f81f0ddf5b.jpg,8
772,50c46e531f7b08351275d5b36993dbbf.jpg,4,3,keloid,inflammatory,non-neoplastic,NaN,34,keloid/50c46e531f7b08351275d5b36993dbbf.jpg,12
1731,649a62d188bad59afe9eb4ecb56fb25e.jpg,3,2,mycosis-fungoides,malignant-cutaneous-lymphoma,malignant,NaN,34,mycosis-fungoides/649a62d188bad59afe9eb4ecb56f...,15
387,23baf39444239889e494ab51aa8bee57.jpg,2,2,acne-vulgaris,inflammatory,non-neoplastic,NaN,12,acne-vulgaris/23baf39444239889e494ab51aa8bee57...,1
...,...,...,...,...,...,...,...,...,...,...
163,2feaf7dad8de6bbe03470747caf74790.jpg,1,4,dermatomyositis,inflammatory,non-neoplastic,NaN,12,dermatomyositis/2feaf7dad8de6bbe03470747caf747...,6
2366,567224dca06e7f3fa5ec45ee4745a4e1.jpg,3,3,squamous-cell-carcinoma,malignant-epidermal,malignant,NaN,34,squamous-cell-carcinoma/567224dca06e7f3fa5ec45...,19
252,30fd701054b889dcde3c3632585fd523.jpg,2,1,basal-cell-carcinoma,malignant-epidermal,malignant,NaN,12,basal-cell-carcinoma/30fd701054b889dcde3c36325...,3
2356,42bc6a8ccf4a8217798fa5545be2b561.jpg,2,4,actinic-keratosis,malignant-epidermal,malignant,NaN,12,actinic-keratosis/42bc6a8ccf4a8217798fa5545be2...,2


In [46]:
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory=data_dir,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)
val_generator = val_datagen.flow_from_dataframe(
    val_data,
    directory=data_dir,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)
#generators for training and validation data
#not using categorical because the model will be trained using xgboost

Found 2288 validated image filenames.
Found 572 validated image filenames.


In [47]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #using VGG16 as the base model
x = base_model.output #adding a global average pooling layer
x = GlobalAveragePooling2D()(x) #flattening the output
model = Model(inputs=base_model.input, outputs=x) #creating the model

In [48]:
train_features = model.predict(train_generator) #extracting features from the training data
val_features = model.predict(val_generator) #extracting features from the validation data

c:\Users\kabir\.vscode\Code Files\AJL-Team-Biotin-BTT-Spring-Studio-\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


72/72 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step


In [49]:
y_train = train_data['encoded_label'] #labels for the training data
y_val = val_data['encoded_label'] #labels for the validation data

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss') #using xgboost as the classifier, using mlogloss as the evaluation metric
xgb_model.fit(train_features, y_train) #fitting the xgboost model on the training features and labels

c:\Users\kabir\.vscode\Code Files\AJL-Team-Biotin-BTT-Spring-Studio-\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:05:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [50]:
y_pred = xgb_model.predict(val_features) #predicting the labels for the validation features

# Generate the classification report
print(classification_report(y_val, y_pred, target_names=label_encoder.classes_)) #printing classification report. hopefully you should see good results!

                                    precision    recall  f1-score   support

                              acne       0.28      0.20      0.23        25
                     acne-vulgaris       0.47      0.68      0.55        40
                 actinic-keratosis       0.50      0.21      0.29        24
              basal-cell-carcinoma       0.35      0.59      0.44        59
  basal-cell-carcinoma-morpheiform       0.20      0.14      0.17         7
                    dermatofibroma       0.20      0.09      0.12        11
                   dermatomyositis       0.40      0.33      0.36        24
                dyshidrotic-eczema       0.00      0.00      0.00         8
                            eczema       0.31      0.31      0.31        26
                   epidermal-nevus       0.29      0.13      0.18        15
                      folliculitis       0.54      0.67      0.60        55
                    kaposi-sarcoma       0.44      0.16      0.24        25
           

In [51]:
#making a test generator
import os
#test_df['file_path'] = test_df['md5hash'].apply(lambda x: os.path.join(test_dir, x)) #this is not needed since we can assume md5hash is in the directory already
def preprocess_test_data(test_df, directory):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=directory,
        x_col='md5hash',
        y_col=None,
        target_size=(128, 128),
        batch_size=32,
        shuffle=False,
        class_mode=None
    )
    return test_generator
test_generator = preprocess_test_data(test_df, test_dir) #preprocessing the test data

Found 1227 validated image filenames.


In [52]:
test_df #didnt need file_path since we are using md5hash directly

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,qc,ddi_scale
0,0844ae634f0e6e7ef1f73c2aeecbae0e.jpg,2,2,NaN,12
1,3b290d262098f761d719aa07cf36c040.jpg,4,3,NaN,34
2,cf561d08ac46d0fda678bff6621005ee.jpg,2,3,NaN,12
3,e6371069be05c6b0a95b4b3f1bacc9a5.jpg,4,3,NaN,34
4,f76cddb37265f97508f159078dcc7e7c.jpg,5,5,NaN,56
...,...,...,...,...,...
1222,270a217e6e961cf405a4ad46a110ff69.jpg,1,1,NaN,12
1223,176892dfc10eabf4ad8b1f50ec5df7e5.jpg,5,6,NaN,56
1224,91a4f52d7f99a33ec53d6d695e6c9f4d.jpg,2,1,NaN,12
1225,8ba5d48405d01accc3a218693a049ae9.jpg,4,2,NaN,34


In [53]:
print(test_features.shape)


(1227, 512)


In [54]:
pre_built_model.summary() #summary of the pre-trained model

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 21)             │         1,365 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,121,665 (42.43 MB)

 Trainable params: 3,707,221 (14.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,414,444 (28.28 MB)

In [55]:
# Manually create a new input layer that matches the original
input_layer = Input(shape=(128, 128, 3))

# Rebuild the same layers up to Dense(512)
x = pre_built_model.layers[0](input_layer)  # Conv2D
x = pre_built_model.layers[1](x)            # MaxPooling2D
x = pre_built_model.layers[2](x)            # Conv2D
x = pre_built_model.layers[3](x)            # MaxPooling2D
x = pre_built_model.layers[4](x)            # Flatten
x = pre_built_model.layers[5](x)            # Dropout
x = pre_built_model.layers[6](x)            # Dense(64)
x = Dense(512, activation='relu')(x)
# Create the feature extractor model
feature_extractor = Model(inputs=input_layer, outputs=x)

In [56]:
test_features = feature_extractor.predict(test_generator, verbose=1)
print(test_features.shape)

c:\Users\kabir\.vscode\Code Files\AJL-Team-Biotin-BTT-Spring-Studio-\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step
(1227, 512)


In [57]:
xgb_preds = xgb_model.predict(test_features)
xgb_preds 

array([10, 19, 10, ..., 19, 10,  0])

In [58]:
train_df['encoded_label'] #we can see our encoded labels here

0       16
1        4
2       12
3        3
4       16
        ..
2855    10
2856    20
2857     0
2858     8
2859    12
Name: encoded_label, Length: 2860, dtype: int64

In [60]:
test_df['predicted_label'] = xgb_preds #assigning the predicted labels to the test_df
test_df['predicted_label'] = label_encoder.inverse_transform(xgb_preds)
test_df['md5hash'] = test_df['md5hash'].str.replace('.jpg', '', regex=False)
test_df[['md5hash', 'predicted_label']].to_csv('submission.csv', index=False) #saving the submission file